In [1]:
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
import scipy.io
import torch.utils.data as Data
import time
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score,roc_curve,auc,average_precision_score,precision_recall_curve

In [2]:
import model 

In [10]:
DATA_PATH = 'deepsea_train/'
device = 'cuda'
BATCH_SIZE = 100

### Loading test data

In [8]:
test_data_raw = scipy.io.loadmat(DATA_PATH+'test.mat')
x_test = torch.FloatTensor(test_data_raw['testxdata'])
y_test = torch.FloatTensor(test_data_raw['testdata'])

# Deep Sea Predictions

In [9]:
deep_sea = model.deep_sea_nn()
deep_sea.load_state_dict(torch.load('deep_sea_model_params.pth'))
deep_sea.to(device)
loss_func = nn.BCEWithLogitsLoss()
print(deep_sea)

deep_sea_nn(
  (conv1): Conv1d(4, 320, kernel_size=(8,), stride=(1,))
  (conv2): Conv1d(320, 480, kernel_size=(8,), stride=(1,))
  (conv3): Conv1d(480, 960, kernel_size=(8,), stride=(1,))
  (maxpool): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.2)
  (drop2): Dropout(p=0.5)
  (linear1): Linear(in_features=50880, out_features=925, bias=True)
  (linear2): Linear(in_features=925, out_features=919, bias=True)
)


In [11]:
test_loader = Data.DataLoader( dataset=Data.TensorDataset(x_test, y_test),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    drop_last=False,
)

In [20]:
print('starting testing')

test_losses = []
y_pred = np.zeros([455024, 919])
running_test_loss = 0
deep_sea.eval()

start_time = time.time()
print('#### Testing started  ...')
for step, (seq, label) in enumerate(test_loader):
    x, y = seq.to(device), label.to(device)
    
    test_out = deep_sea.foward(x)
    test_loss = loss_func(test_out, y)

    running_test_loss = test_loss.item()
    test_losses.append(running_test_loss)
            
    test_output = torch.sigmoid(test_out.cpu().data)
    
    if(step<4550):
        for i, j in zip(range(step*100, (step+1)*100),range(0, 100)):
            y_pred[i, :] = test_output.numpy()[j, :]
    else:
        for i,j in zip(range(455000,455024),range(0,24)):
            y_pred[i, :] = test_output.numpy()[j, :]
    
    
time_elapsed = time.time() - start_time
print('#### Testing time elapsed:', time_elapsed)

starting testing
#### Testing started  ...
#### Testing time elapsed: 62.71815347671509


In [22]:
## Saving predictions
np.save('pred_deep_sea_model_20191022',y_pred)